# Лабораторная работа №2
## Реализация численных методов решения СЛАУ.
---  
### Цель: Оценивание невязок некоторых численных методов решения СЛАУ. Определение числа обусловленности матрицы и критерия останова итераций метода верхней релаксации
---
#### Для исследования выберем СЛАУ вида (пункт 'к'): 


$$
\begin{cases} 
a_{11}x_{11} + a_{12}x_{12} + ... + a_{1n}x_{1n} = f_{1} \\
...~~~~~~~~~~~~~~~~~~~~~~~...~~~~~~~~~~~~~~~~~~...\\
a_{n1}x_{n1} + a_{n2}x_{n2} + ... + a_{nn}x_{nn} = f_{n}
\end{cases}
$$
##### При $\displaystyle n = 10, a_{ii} = \frac{1}{i + j} (i \neq j, f_i = \frac{1}{i})$


### Для выполнения лабораторной работы остановимся на выборе трёх эквивалентных норм:

$||x||_1 = \max\limits_{k} |x_k|$

$||A||_1 = \max\limits_{1 \leq i \leq n} \sum\limits_{1 \leq j \leq n}^{n} |a_{ij}|$ 

---

$||x||_2 = \sum\limits_{k} |x_k|$

$||A||_2 = \max\limits_{1 \leq j \leq n} \sum\limits_{1 \leq i \leq n}^{n} |a_{ij}|$ 

---

$||x||_3 = \sqrt{(x, x)}$

$||A||_3 = \sqrt{|\max\limits_{1 \leq i \leq n} \lambda^{i} \cdot (A^{*} A)|}$

--- 

#### Испорт модулей:

In [ ]:
import numpy as np
from abc import ABC, ABCMeta, abstractmethod, abstractproperty

#### Объявление классов: 

In [ ]:
######           Class norm

class INorm (ICallable, ABC):
    norm = 0

    def __init__(self, norm):
        self.norm = norm
        
class VecNorm (INorm):

    def __call__(self, vec):
        return self.norm(vec)

class MatrixNorm (INorm):

    def __call__(self, matrix):
        return self.norm(matrix)

#### Добавление вспомогательных функций для дальнейшего исследования: 

In [ ]:
def find_max_eig (matrix):
    dim = matrix.shape[0]
    x = np.random.rand(dim)
    num_iters = 100
    for i in range(num_iters):
        x = np.dot(matrix, x) / VecNorms[0](x)
    return np.dot(np.dot(x.T, matrix), x) / np.dot(x.T, x)

def find_min_eig (matrix):
    return 1 / find_max_eig(np.linalg.inv(matrix))

def is_LU_possible(matrix):
    dim = matrix.shape[0]
    for i in range(dim):
        if np.linalg.det(matrix[:i, :i]) == 0:
            return False
    return True    

def get_LU (matrix):
    dim = matrix.shape[0]
    L_part = np.eye(dim)
    U_part = np.zeros((dim, dim))
    
    for i in range(dim):
        for j in range(dim):
            if i > j:
                L_part[i][j] = (matrix[i][j] - sum(np.dot(L_part[i:i+1, 0:j], U_part[0:j, j:j+1]))) / U_part[j][j]
            else:
                U_part[i][j] = matrix[i][j] - sum(np.dot(L_part[i:i+1, 0:i], U_part[0:i, j:j+1]))
    return L_part, U_part

def get_l_solution(L_part, rhs):
    sol = np.zeros((L_part.shape[0], 1))
    sol[0] = rhs[0] / L_part[0][0]
    for i in range(1, len(sol)):
        sol[i] = (rhs[i] - sum(np.dot(L_part[i:i+1, :i], sol[:i]))) / L_part[i][i]
    return sol

def get_u_solution(U_part, sol):
    U_part_dim = U_part.shape[0]

    x = np.zeros((U_part_dim, 1))
    x[-1] = sol[-1] / U_part[-1][-1]
    for i in range(U_part_dim - 1, -1, -1):
        x[i] = (sol[i] - sum(np.dot(U_part[i:i+1, i+1:], x[i+1:]))) / U_part[i][i]
    return x

def over_relaxation_is_available(matrix):
    for i in range(matrix.shape[0]):
        if np.linalg.det(matrix[:i, :i]) <= 0:
            return False
    return True

def over_relaxation(matrix, rhs, epsilon=1e-6):
    dim = matrix.shape[0]
    
    D_part = np.zeros([dim, dim])
    L_part = np.zeros([dim, dim])
    U_part = np.zeros([dim, dim])
    
    for i in range(0, dim):
        for j in range(0, dim):
            if i < j:
                U_part[i, j] = matrix[i, j]
            elif i > j:
                L_part[i, j] = matrix[i, j]
            else:
                D_part[i, j] = matrix[i, j]
    
    w = np.random.rand() + 1.01 # the random number in range (1, 2)
    tmp = np.linalg.inv(w * L_part + D_part)
    u_k = np.matmul(-tmp, D_part * (w - 1) + U_part * w)
    
    num_iters = 0
    x = (np.random.rand(1, dim)).T
    norm = VecNorms[2]
    while norm(rhs - np.matmul(matrix, x)) >= epsilon:
        x = np.matmul(w * tmp, rhs) + np.matmul(u_k, x) 
        num_iters += 1

    return (x, num_iters)

#### Инициализация объектов, используемых в исследованиии 

In [ ]:
dim = 10
rhs = np.array([[1 / i] for i in range(1, dim + 1)])
matrix = np.zeros((dim, dim)) 

for i in range(dim): # Fill the matrix from point 'k'
    for j in range(dim):
        if i == j:
            matrix[i, j] = 1
        else:
            matrix[i, j] = 1 / (i + j + 2)

VecNorms = [] #Norm for vectors
VecNorms.append(VecNorm(lambda vec: np.abs(vec).max()))
VecNorms.append(VecNorm(lambda vec: np.abs(vec).sum()))
VecNorms.append(VecNorm(lambda vec: np.sqrt(np.dot(vec.T, vec)).item()))

MatrixNorms = [] #Norm for matrixes
MatrixNorms.append(MatrixNorm(lambda matrix: np.abs(np.abs(matrix).sum(axis=1, dtype='float')).max()))
MatrixNorms.append(MatrixNorm(lambda matrix: np.abs(np.abs(matrix).sum(axis=0, dtype='float')).max()))
MatrixNorms.append(MatrixNorm(lambda matrix: np.sqrt(find_max_eig(np.matmul(matrix.T, matrix)))))

#### Основная часть

In [ ]:
def main ():

    eigenvec = np.linalg.eigvals(matrix)

    real_min_eigval = min(eigenvec)
    real_max_eigval = max(eigenvec)

    iter_max_eigval = find_max_eig(matrix); 
    iter_min_eigval = find_min_eig(matrix)

    print ('Iteratively calculated max eigenvalue: ' + str(real_max_eigval))
    print ('Iteratively calculated min eigenvalue: ' + str(real_min_eigval))

    print('Error of iterative method of computing max eigenvalue: ' + \
           str(np.abs((iter_max_eigval - real_max_eigval))))

    print('Error of iterative method of computing min eigenvalue: ' + \
           str(np.abs((iter_min_eigval - real_min_eigval))))

    print ("--------------------------")

    for i in range (3):
        print ('Condition number with matrix norms #' + str(i) + ': ' + \
               str(MatrixNorms[i](matrix) * MatrixNorms[i](np.linalg.inv(matrix))))

    print ("--------------------------")

    print ('Is LU decomposition is possible?')
    if (is_LU_possible(matrix)):
        print ('\t Yes, it is')
    else:
        print ('\t No, it isn\'t')

    L, U = get_LU(matrix)
    y = get_l_solution(L, rhs)
    ans = get_u_solution(U, y)
    print("The solution via LU-decompisition:")
    print(ans)

    error_vec = np.abs(np.matmul(matrix, ans) - rhs)
    print("Errors with different norms:")
    for i in range (3):
        print ('\tThe ' + str(i + 1) + 'th norm of error\'s vector: ' + str(VecNorms[i](error_vec)))

    print ("--------------------------")

    print ('Is over relaxation is possible?')
    if (over_relaxation_is_available(matrix)):
        print ('\t Yes, it is')
    else:
        print ('\t No, it isn\'t')
    
    solution, num_iters = over_relaxation(matrix, rhs)
    print("The solution via over relaxation:")
    print(solution)
    print("Number of iterations: " + str(num_iters))

    error_vec = np.abs(np.matmul(matrix, solution) - rhs)
    print("Errors with different norms:")
    for i in range (3):
        print ('\tThe ' + str(i + 1) + 'th norm of error\'s vector: ' + str(VecNorms[i](error_vec)))

main()

---
### Результаты работы программы:
```
Iteratively calculated max eigenvalue: 2.048359926977445
Iteratively calculated min eigenvalue: 0.6579597538101791
Error of iterative method of computing max eigenvalue: 1.3322676295501878e-15
Error of iterative method of computing min eigenvalue: 7.771561172376096e-16
--------------------------
Condition number with matrix norms #0: 5.6336089382284
Condition number with matrix norms #1: 5.6336089382284005
Condition number with matrix norms #2: 3.1131994246085664
--------------------------
Is LU decomposition is possible?
	 Yes, it is
The solution via LU-decompisition:
[[ 9.19077109e-01]
 [ 1.75540170e-01]
 [ 6.39348240e-02]
 [ 2.72747640e-02]
 [ 1.14234685e-02]
 [ 3.51083928e-03]
 [-7.89957814e-04]
 [-3.25080145e-03]
 [-4.69787781e-03]
 [-5.55373994e-03]]
Errors with different norms:
	The 1th norm of error's vector: 5.551115123125783e-17
	The 2th norm of error's vector: 2.0816681711721685e-16
	The 3th norm of error's vector: 1.0103182026100663e-16
--------------------------
Is over relaxation is possible?
	 Yes, it is
The solution via over relaxation:
[[ 9.19076482e-01]
 [ 1.75540068e-01]
 [ 6.39348855e-02]
 [ 2.72748748e-02]
 [ 1.14235879e-02]
 [ 3.51095127e-03]
 [-7.89859234e-04]
 [-3.25071805e-03]
 [-4.69780943e-03]
 [-5.55368558e-03]]
Number of iterations: 15
Errors with different norms:
	The 1th norm of error's vector: 5.549383251812756e-07
	The 2th norm of error's vector: 1.1356259986827766e-06
	The 3th norm of error's vector: 6.111299797495808e-07
```

## Заключение
### Для исследуемой СЛАУ получены все интересующие нас характеристики:

---

#### Собственные числа, полученные методом иттераций:
* $\lambda_{max} = 2.048359926977445$ 
* $\lambda_{min} = 0.6579597538101791$ 

Отклонения от реальных значений для 100 итераций: 

* $\delta_{lambda_{max}} = 1.3322676295501878 \cdot 10^{-15}$ 
* $\delta_{lambda_{min}} = 7.771561172376096 \cdot 10^{-16}$ 

Видно, что итеративный метод даёт достаточно точный результат

---

#### Числа обусловленности для 3-х различных норм матрицы:
* $\mu_1 = 5.6336089382284$
* $\mu_2 = 5.6336089382284005$
* $\mu_3 = 3.1131994246085664$

Числа обусловленности получились много меньше сотни, из чего можно сделать вывод, что задача 'хорошо обусловлена'

---

#### Результаты, полученные численными методами решения СЛАУ оказались правильными с высокой степенью точности (невязка $\varepsilon = 10^{-6}$):

Погрешности для LU-разложения (по трём различным нормам):

* $\delta_1 = 5.551115123125783 \cdot 10^{-17}$
* $\delta_2 = 2.0816681711721685 \cdot 10^{-16}$
* $\delta_3 = 1.0103182026100663 \cdot 10^{-16}$

Погрешности для метода верхней релаксации (по трём различным нормам):

* $\delta_1 = 5.549383251812756 \cdot 10^{-7}$
* $\delta_2 = 1.1356259986827766 \cdot 10^{-6}$
* $\delta_3 = 6.111299797495808 \cdot 10^{-6}$

Для последнего метода критерий останова (число итераций) составил $N = 15$

---

### Из проведенной работы можно сделать вывод, что численные методы решения СЛАУ для хорошо обусловленных задач дают результаты высокой точности